# Checking enviroment states and action sets

In [1]:
import sys
sys.path.append("..")
from DDQN_Agent import *

2023-03-23 22:05:21.515969: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
if os.name == 'nt':
    main_hyper_dir = "D:\\Artificial_Intelligence\\Portfolio\\RL_updated\\MountainCar\\" # Windows
    conda_python_exec = 'C:\\Users\\yanie\\anaconda3\\envs\\ai_dev\\python.exe '# Windows
    logs_dir = main_hyper_dir+'Hyperparam_tuning\\' # Windows
else:
    main_hyper_dir = "/media/n/NewDisk/Artificial_Intelligence/Portfolio/RL_updated/MountainCar/" # Linux
    conda_python_exec = '/home/n/anaconda3/envs/ai_dev/bin/python '# Linux
    logs_dir = main_hyper_dir+'Hyperparam_tuning_ddqn/' # Linux

ENV = "MountainCar-v0"
SUCESS_CRITERIA_VALUE = -100
SUCESS_CRITERIA_EPOCH = 100
EPISODES = 1000   

In [3]:
env = gym.make(ENV)#,new_step_api=True
env

<TimeLimit<OrderEnforcing<PassiveEnvChecker<MountainCarEnv<MountainCar-v0>>>>>

In [4]:
s = env.observation_space.sample()
s, s.shape

(array([-0.19794798,  0.05583682], dtype=float32), (2,))

In [5]:
env.observation_space, env.observation_space.shape

(Box([-1.2  -0.07], [0.6  0.07], (2,), float32), (2,))

In [6]:
s = env.reset()
env.action_space.n, env.action_space, env.step(1)

(3,
 Discrete(3),
 (array([-4.6636066e-01, -4.3035913e-04], dtype=float32),
  -1.0,
  False,
  False,
  {}))

### Training/Hyperparam run

In [7]:
%tensorboard --logdir logs_general/hyper

UsageError: Line magic function `%tensorboard` not found.


In [8]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "Training/fit_DDQN/"


if TUNING_TYPE == "MANUAL":
    params = {}
    params["discount"] =[0.94, 0.99]
    params["end_of_episode"] = [400]
    params["learning_rate"] = [1e-6, 0.001]
    params["tau_update_network"] = [0.001, 1e-4]
    params["exploration_technique"] = [ 'epsilon', 'boltzman' ]
    params["training_steps"] = [3000000]
    params["dense_units"] = [74,32]
    params["time_to_update"] = [500] #400,

    hyperparam_combination = list(itertools.product(*list(params.values())))

    try:

        files = [name for name in os.listdir(logs_dir) if os.path.isfile(os.path.join(logs_dir, name)) and name != "logfile.txt" and name != "merged_results.json"]
        if len(files) >= 1: merge_JsonFiles(main_hyper_dir, logs_dir, files)

        res_file = logs_dir+"merged_results.json"
        def without_keys(d, keys):
            return {x: d[x] for x in d if x not in keys}


        if os.path.isfile(res_file):
            with open(res_file, 'r') as f:
                complete_file = json.load(f)

            newlist = sorted(complete_file, key=lambda d: d['mean_rewards'], reverse=True) 
            params = []
            for i, f in enumerate(newlist):
                label = "disc : "+str(f['discount'])+" | "+"lr : "+str(f['learning_rate'])+" | "+"entropy : "+str(f['entropy_factor'])+" | "+"update : "+str(f['time_to_update'])
                plt.figure(figsize=[35,4])
                plt.plot(f['rewards'], label = label)
                plt.legend()
                max_mean_reward = f['mean_rewards']
                params.append(f)
                print(without_keys(f,"rewards"))
                if i == 10:
                    break
            plt.title("Evaluation rewards"); plt.grid();
            plt.show()

    except:
        pass

    

In [9]:
if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_DDQN"
    project_name = "keras_tunning_epsilon"
    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/DDQN_epsilon/", exploration_tech ='epsilon' ),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 50,
            # distribution_strategy=tf.distribute.MirroredStrategy(),
            directory= dir,
            project_name=project_name
        )
    tuner.search(x=[0], y=[1])

    # project_name = "keras_tunning_boltzman"
    # tuner = kt.BayesianOptimization(
    #         MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyperV2/DDQN_boltzman/", exploration_tech ='boltzman' ),
    #         objective= kt.Objective('total_train_reward', direction="max"), 
    #         max_trials = 20,
    #         # distribution_strategy=tf.distribute.MirroredStrategy(),
    #         directory= dir,
    #         project_name=project_name
    #     )
    # tuner.search(x=[0], y=[1])
   
else : 
    print("Acquiring parameters ....")
    # run_training(training_steps, learning_rate, tau_update_network, exploration_tech, discount, time_to_update, dense_units, writer, end_of_episode)
        

INFO:tensorflow:Reloading Tuner from Hyperparam_kt_DDQN/keras_tunning_epsilon/tuner0.json
INFO:tensorflow:Oracle triggered exit


In [10]:
dir = r"Hyperparam_kt_DDQN"
exploration_tech="epsilon"

project_name = "keras_tunning_epsilon"
tuner = kt.BayesianOptimization(
        MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/DDQN_epsilon/", exploration_tech ='epsilon' ),
        objective= kt.Objective('total_train_reward', direction="max"), 
        max_trials = 50,
        # distribution_strategy=tf.distribute.MirroredStrategy(),
        directory= dir,
        project_name=project_name
    )

hyperparam_combination =[]
for trials in tuner.oracle.get_best_trials(num_trials=3):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

    disc = trials.hyperparameters.values["discount"]
    end_ep = 1000
    train_steps = 5000000
    d = trials.hyperparameters.values["dense_units"]
    lr = trials.hyperparameters.values["learning_rate"]
    time_to_update = trials.hyperparameters.values["time_to_update"]
    tau_update_network = trials.hyperparameters.values["tau_update_network"]

    hyperparam_combination.append((disc, end_ep, lr, tau_update_network, exploration_tech, train_steps, d, time_to_update))


INFO:tensorflow:Reloading Tuner from Hyperparam_kt_DDQN/keras_tunning_epsilon/tuner0.json
Trial id :37 | Score :-135.6 --> {'discount': 0.99, 'learning_rate': 0.004943338266926713, 'time_to_update': 800, 'dense_units': 116, 'tau_update_network': 0.1}
Trial id :18 | Score :-136.02 --> {'discount': 0.99, 'learning_rate': 0.0030296137455032425, 'time_to_update': 800, 'dense_units': 160, 'tau_update_network': 0.025184133983643338}
Trial id :33 | Score :-137.36 --> {'discount': 0.99, 'learning_rate': 0.0031508505504027704, 'time_to_update': 700, 'dense_units': 109, 'tau_update_network': 0.1}


In [11]:
len(hyperparam_combination), hyperparam_combination

(3,
 [(0.99, 1000, 0.004943338266926713, 0.1, 'epsilon', 5000000, 116, 800),
  (0.99,
   1000,
   0.0030296137455032425,
   0.025184133983643338,
   'epsilon',
   5000000,
   160,
   800),
  (0.99, 1000, 0.0031508505504027704, 0.1, 'epsilon', 5000000, 109, 700)])

In [12]:
# run_hyperparam(total_files_number = 4, TUNING_TYPE= "MANUAL",  logs_dir=logs_dir,  main_hyper_dir = main_hyper_dir, conda_python_exec= conda_python_exec, py_file = "mountaincar_DDQN.py" ,TUNING_TYPE = TUNING_TYPE, hyperparam_combination = hyperparam_combination)

In [13]:
val_env = gym.make(ENV, render_mode = "rgb_array")
dir = r"Hyperparam_kt_DDQN"

for exploration_tech in ["epsilon", "boltzman"]:

    project_name = "keras_tunning_"+exploration_tech
    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/DDQN_"+exploration_tech+"/", exploration_tech =exploration_tech),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 50,
            directory= dir,
            project_name=project_name
        )

    for trials in tuner.oracle.get_best_trials(num_trials=1):
        print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)
        

    env_model = tuner.get_best_models()[0]
    final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./DDQN_"+exploration_tech+"_video.mp4")
    print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))

    

INFO:tensorflow:Reloading Tuner from Hyperparam_kt_DDQN/keras_tunning_epsilon/tuner0.json
Trial id :37 | Score :-135.6 --> {'discount': 0.99, 'learning_rate': 0.004943338266926713, 'time_to_update': 800, 'dense_units': 116, 'tau_update_network': 0.1}
Trial number :  50
Moviepy - Building video ./DDQN_epsilon_video.mp4.
Moviepy - Writing video ./DDQN_epsilon_video.mp4



Moviepy - Done !
Moviepy - video ready ./DDQN_epsilon_video.mp4


Final mean reward ' epsilon ': -138.6
INFO:tensorflow:Reloading Tuner from Hyperparam_kt_DDQN/keras_tunning_boltzman/tuner0.json
Trial id :20 | Score :-124.56 --> {'discount': 0.99, 'learning_rate': 0.001763393784940077, 'time_to_update': 600, 'dense_units': 299, 'tau_update_network': 0.07370751378110506}
Trial number :  50
Moviepy - Building video ./DDQN_boltzman_video.mp4.
Moviepy - Writing video ./DDQN_boltzman_video.mp4



Moviepy - Done !
Moviepy - video ready ./DDQN_boltzman_video.mp4


Final mean reward ' boltzman ': -124.92
